# Leakage Detection Tests

A series of tests for leakage, demonstrated on the Los Angeles Chronic Offenders database  
Requires a credentials.py file defining the following variables: dbname, user, host, password

In [1]:
import sys
import os
import math
import warnings

import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from IPython.core.interactiveshell import InteractiveShell

from credentials import dbname, user, host, password

warnings.filterwarnings(action='once')
InteractiveShell.ast_node_interactivity = "all"

/mnt/data/users/jwang/venv3/lib/python3.4/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def execute_sql(statement, dbname, user, host, password, isolation = False, results = True):
    """
    Use psycopg2 to execute PostgreSQL queries
    
    Input:
        statement (str): SQL statement to run in database
        dbname, user, host, password (str): database credentials
        isolation (bool): indicator for whether to change isolation level to autocommit; True for queries that cannot be run 
            from within a transation (see https://wiki.postgresql.org/wiki/Psycopg2_Tutorial)
        results (bool): indicator for whether the query is expected to output results;
            for example, True for SELECT statements and False for CREATE TABLE statements
    
    Output:
        relation (dataframe): query results or empty dataframe if results = False
    """
    conn = psycopg2.connect("dbname={} user={} host={} password={}".format(dbname, user, host, password))
    cur = conn.cursor()
    if isolation:
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur.execute(statement)
    relation = pd.DataFrame()
    if results:
        results = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        relation = pd.DataFrame(results, columns=colnames)
    cur.close()
    conn.close()
    return relation


## I. Create a randomized version of the raw schema

Pull the information schema from selected database.

In [ ]:
# name of the database schema containing raw data to be randomized
RAW_SCHEMA = 'raw' 

In [4]:
statement = "SELECT * FROM information_schema.tables;"
tables = execute_sql(statement, dbname, user, host, password)

In [5]:
tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,los_angeles_chronic_offender,ccms,cases,BASE TABLE,None,None,None,None,None,YES,NO,None
1,los_angeles_chronic_offender,ccms,addinfo,BASE TABLE,None,None,None,None,None,YES,NO,None
2,los_angeles_chronic_offender,ccms,casesaka,BASE TABLE,None,None,None,None,None,YES,NO,None
3,los_angeles_chronic_offender,ccms,casescharge,BASE TABLE,None,None,None,None,None,YES,NO,None
4,los_angeles_chronic_offender,ccms,casesflag,BASE TABLE,None,None,None,None,None,YES,NO,None
5,los_angeles_chronic_offender,ccms,citation,BASE TABLE,None,None,None,None,None,YES,NO,None
6,los_angeles_chronic_offender,kit_test,citation,BASE TABLE,None,None,None,None,None,YES,NO,None
7,los_angeles_chronic_offender,dedupe_cases,entities,BASE TABLE,None,None,None,None,None,YES,NO,None
8,los_angeles_chronic_offender,ccms,status_h,BASE TABLE,None,None,None,None,None,YES,NO,None
9,los_angeles_chronic_offender,ccms,prob,BASE TABLE,None,None,None,None,None,YES,NO,None


Select the schema containing raw datasets to be randomized. For this tutorial, the schema is named ???

In [6]:
raw = tables[tables.table_schema == RAW_SCHEMA]
raw.head()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action


First created the _randomized schema if it doesn't yet exist

In [ ]:
statement = "CREATE SCHEMA IF NOT EXISTS {}_randomized;".format(RAW_SCHEMA)
output = execute_sql(statement, dbname, user, host, password, port, isolation=True, results = False)

Randomize every table in raw and write output to _randomized schema

In [ ]:
for table_name in raw.table_name:
    print("Working on table {}".format(table_name))
    
    # Pull the table from original schema
    print("\tPulling table")
    statement = "SELECT * FROM {}.{};".format(RAW_SCHEMA, table_name)
    table = execute_sql(statement, dbname, user, host, password, port)
    
    # Randomize the table
    print("\tRandomizing")
    randomized_table = randomize(table)
    
    # Make a new table in _randomized schema
    print("\tUploading randomized version")
    statement = "CREATE TABLE IF NOT EXISTS {}_randomized.{} (LIKE {}.{});".format(RAW_SCHEMA, table_name, RAW_SCHEMA, table_name)
    output = execute_sql(statement, dbname, user, host, password, port, isolation = True, results = False)
    
    # Write results into new table
    statement = "SELECT COUNT(*) FROM {}_randomized.{};".format(RAW_SCHEMA, table_name)
    output = execute_sql(statement, dbname, user, host, password, port)
    if output.iloc[0,0]>0: #  do nothing if new table already contains data
        print("\t*****SKIPPING TABLE {} -- it already has data".format(table_name))
    else:
        engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))
        randomized_table.to_sql(table_name, engine, schema = '{}_randomized'.format(RAW_SCHEMA), index = False, if_exists='append')
    

## II. Edit + execute preprocessing and config files so that they point to the newly randomized raw schema

New versions of the preprocessing and config files with an "_edited" suffix will be produced.

## III. Run an experiment with this new setup

Now that we've set up randomized versions of the original schemas in the database and created a new config file for a randomized experiment, it's time to run the actual triage experiment.

**NOTE**: This must be done from the command line!

- First validate the new config file. For this tutorial: ./tutorial.sh triage --config_file inspections_test_edited.yaml validate
- Then execute the experiment. For this tutorial: ./tutorial.sh triage --config_file inspections_test_edited.yaml run

## IV. Align the results of randomized experiment against original experiment

First, we need to map the model groups from our new, randomized experiment to the model groups from the original experiment

Confirm that we now see a set of model groups where purpose = "leakage-detection"

In [7]:
statement = "SELECT * FROM results.model_groups;"
model_groups = execute_sql(statement, dbname, user, host, password, port)
model_groups

NameError: name 'port' is not defined

Recall that a model group is defined by the classifier (model_type), its parameters (model_parameters), the features (feature_list), and the model_config. Since we varied the "purpose" key within model_config for these randomized experiments, all other properties of a model group should be used as index keys. This includes other model group keys in the model_config column.

In [ ]:
# pull out all model group keys from model_config column and make new columns from them
list_of_lists = list(model_groups.model_config.apply(lambda x: list(x.keys())))
keys = set([key for x in list_of_lists for key in x])

for key in keys:
    model_groups[key] = model_groups.model_config.apply(lambda x: x.get(key, None))

In [ ]:
# convert model_parameters and feature_list columns to strings so they can be used as index keys
model_groups['model_parameters'] = model_groups.model_parameters.apply(lambda x: ', '.join('{}{}'.format(key, val) for key, val in x.items()))
model_groups['feature_list'] = model_groups.feature_list.apply(lambda x: ', '.join(x))

In [ ]:
index_cols = [col for col in model_groups.columns if col not in ['purpose', 'model_group_id', 'model_config']]
index_cols

In [ ]:
model_groups = model_groups.set_index(index_cols)

Join the original and randomized model groups to create a wide (rather than long) version of model_group table. Each row of model_groups_wide corresponds to a particular model group configuration and indicates the model group ids of the original vs randomized versions.

In [ ]:
# define original "purpose" model group key
ORIG_PURPOSE = 'test'

In [ ]:
orig = model_groups[model_groups.purpose == ORIG_PURPOSE]
rand = model_groups[model_groups.purpose == 'leakage-detection']
model_groups_wide = pd.merge(orig[['model_group_id']], rand[['model_group_id']], how='right', left_index=True, right_index=True, suffixes = ['_orig', '_rand'])

In [ ]:
model_groups_wide

Pick a model group configuration on which to test for leakage. Let's go with the first row, where model_group_id_orig = 1, model_group_id_rand = 7

In [ ]:
# create variables for the model_group id's you selected
GROUP_ID_ORIG = 1
GROUP_ID_RAND = 7

Look at the models relevant to each model group

In [ ]:
statement = "SELECT * FROM results.models where model_group_id = {};".format(GROUP_ID_ORIG)
models_orig = execute_sql(statement, dbname, user, host, password, port)
models_orig

In [ ]:
statement = "SELECT * FROM results.models where model_group_id = {};".format(GROUP_ID_RAND)
models_rand = execute_sql(statement, dbname, user, host, password, port)
models_rand


Pick a pair of comparable models on which to test for leakage, i.e. with matching train_end_time. Let's go with the first row of each table, i.e. model_id numbers 1 (original) and 13 (random).  
Grab the predictions made by each of these models.

In [ ]:
# create variables for the model id's you selected
MODEL_ID_ORIG = 1
MODEL_ID_RAND = 13

In [ ]:
statement = "SELECT * FROM results.predictions where model_id = {};".format(MODEL_ID_ORIG)
predictions_orig = execute_sql(statement, dbname, user, host, password, port)
predictions_orig

In [ ]:
predictions_orig_sorted = predictions_orig.sort_values(by=['score', 'label_value'], ascending=[False, True])

In [ ]:
statement = "SELECT * FROM results.predictions where model_id = {};".format(MODEL_ID_RAND)
predictions_rand = execute_sql(statement, dbname, user, host, password, port)
predictions_rand

In [ ]:
predictions_rand_sorted = predictions_rand.sort_values(by=['score', 'label_value'], ascending=[False, True])

Grab the evaluation metrics for each of these models.

In [ ]:
statement = "SELECT * FROM results.evaluations where model_id = {};".format(MODEL_ID_ORIG)
evaluations_orig = execute_sql(statement, dbname, user, host, password, port)
evaluations_orig

In [ ]:
statement = "SELECT * FROM results.evaluations where model_id = {};".format(MODEL_ID_RAND)
evaluations_rand = execute_sql(statement, dbname, user, host, password, port)
evaluations_rand

## V. Review the predictions and important features of randomized model